# Demo 5 — Model Serving

Wrap the ARIMA model as a **pyfunc** for serving, then:
1. Serve it locally with `mlflow models serve`
2. Package it as a Docker image with `mlflow models build-docker`

In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import mlflow
import warnings
warnings.filterwarnings("ignore")
from features import fourier_features

In [3]:
mlflow.set_tracking_uri("http://localhost:5050")
mlflow.set_experiment("temperature-forecast-trial")

MODEL_NAME = "temperature-forecast-trial"

## Define a pyfunc wrapper for ARIMA

The native statsmodels model can't be served directly via REST because its `predict()` API doesn't match what MLflow serving expects. We wrap it so that `predict()` accepts a DataFrame with an `n_steps` column, generates Fourier features for the forecast horizon, and returns forecasted values.

In [4]:
class ARIMAWrapper(mlflow.pyfunc.PythonModel):
    """Wraps a fitted statsmodels ARIMA for MLflow serving."""

    def __init__(self, ref_date, last_train_date):
        self.ref_date = ref_date
        self.last_train_date = last_train_date

    def load_context(self, context):
        self.model = mlflow.statsmodels.load_model(context.artifacts["arima_model"])

    def predict(self, context, model_input, params=None):
        from features import fourier_features

        n_steps = int(model_input["n_steps"].iloc[0])
        forecast_dates = pd.date_range(
            start=self.last_train_date + pd.Timedelta(days=1),
            periods=n_steps, freq="D",
        )
        exog = fourier_features(forecast_dates, self.ref_date)
        forecast = self.model.forecast(steps=n_steps, exog=exog)
        return forecast.tolist()

/Users/daniele/miniconda3/envs/ml-gravio/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


## Train, wrap, and register the pyfunc model

In [ ]:
df = pd.read_csv("../data/jena_daily_temp.csv", parse_dates=["Date Time"], index_col="Date Time")
train = df.iloc[:-90]
test = df.iloc[-90:]

ref_date = df.index[0]
exog_train = fourier_features(train.index, ref_date)

# Fit the underlying ARIMA model
order = (5, 1, 2)
model = ARIMA(train["temperature"], order=order, exog=exog_train)
results = model.fit()

Fitted ARIMA(5, 1, 2) — AIC: 12649.09


In [6]:
with mlflow.start_run(run_name="pyfunc-serving") as run:
    # First log the statsmodels model as an internal artifact
    arima_model_info = mlflow.statsmodels.log_model(results, name="statsmodels_arima")

    # Then log the pyfunc wrapper that references it
    pyfunc_model_info = mlflow.pyfunc.log_model(
        name="arima_model",
        python_model=ARIMAWrapper(ref_date=ref_date, last_train_date=train.index[-1]),
        artifacts={"arima_model": arima_model_info.model_uri},
        code_paths=["features.py"],
        input_example=pd.DataFrame({"n_steps": [30]}),
    )

    # Register the pyfunc model
    mv = mlflow.register_model(
        model_uri=f"runs:/{run.info.run_id}/arima_model",
        name=MODEL_NAME,
    )

    # Set @champion alias
    client = mlflow.MlflowClient()
    client.set_registered_model_alias(MODEL_NAME, "champion", mv.version)

    print(f"Registered pyfunc model v{mv.version} with @champion alias")

2026/02/26 16:25:07 WARNING mlflow.pyfunc: Passing a Python object as `python_model` causes it to be serialized using CloudPickle, it requires exercising caution as Python object serialization mechanisms may execute arbitrary code during deserialization.Consider using a file path (str or Path) instead. See https://mlflow.org/docs/latest/ml/model/models-from-code/ for details.
2026/02/26 16:25:07 INFO mlflow.pyfunc: Inferring model signature from input example


Registered model 'temperature-forecast-trial' already exists. Creating a new version of this model...
2026/02/26 16:25:11 WARNING mlflow.tracking._model_registry.fluent: Run with id 413c68019e2b4ad2821520fbcb3775d0 has no artifacts at artifact path 'arima_model', registering model based on models:/m-13dd3c750b1d41db8981d762656b2ed9 instead
2026/02/26 16:25:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: temperature-forecast-trial, version 3
Created version '3' of model 'temperature-forecast-trial'.


Registered pyfunc model v3 with @champion alias
🏃 View run pyfunc-serving at: http://localhost:5050/#/experiments/5/runs/413c68019e2b4ad2821520fbcb3775d0
🧪 View experiment at: http://localhost:5050/#/experiments/5


## Test the pyfunc model locally

In [7]:
loaded = mlflow.pyfunc.load_model(f"models:/{MODEL_NAME}@champion")

result = loaded.predict(pd.DataFrame({"n_steps": [7]}))
print("7-day forecast:", result)

7-day forecast: [13.12626916874401, 13.15917272478445, 13.186079074311833, 12.973600147248318, 12.934278065035015, 12.728458773505682, 12.580103463176556]


---

## Part 1: Serve with `mlflow models serve`

Run this in a terminal:

```bash
MLFLOW_TRACKING_URI=http://localhost:5050 mlflow models serve \
  -m "models:/temperature-forecast-simple@champion" \
  -p 5001 --no-conda
```

> **Note:** We set `MLFLOW_TRACKING_URI` so the CLI can reach the dockerized MLflow server to download the model artifacts.

In [8]:
import requests

payload = {
    "dataframe_split": {
        "columns": ["n_steps"],
        "data": [[30]]
    }
}

response = requests.post(
    "http://localhost:5001/invocations",
    json=payload,
)

print(f"Status: {response.status_code}")
print(f"Forecast: {response.json()}")

Status: 200
Forecast: {'predictions': [13.698623393911678, 14.213954983232066, 14.750731057335202, 14.93293845275739, 15.00506514684565, 15.059875746724265, 15.105628993375925, 15.152345467884409, 15.187955913472166, 15.213018386365713, 15.230453697613873, 15.242637644189825, 15.251652119773329, 15.258374527226302, 15.263364844718144, 15.267036421083583, 15.269713057560685, 15.271666581782412, 15.273095821462917, 15.274143972357361, 15.274913361038113, 15.275477654577916, 15.275891238025856, 15.2761942474753, 15.276416258083682, 15.276578963928538, 15.27669822250601, 15.276785637371207, 15.276849707569882, 15.276896664814437]}


Equivalent curl command:

```bash
curl http://localhost:5001/invocations \
  -H "Content-Type: application/json" \
  -d '{"dataframe_split": {"columns": ["n_steps"], "data": [[30]]}}'
```

---

## Part 2: Build a Docker image

```bash
MLFLOW_TRACKING_URI=http://localhost:5050 mlflow models build-docker \
  -m "models:/temperature-forecast-simple@champion" \
  -n temp-forecast-server
```

Then run:

```bash
docker run -p 5001:8080 temp-forecast-server
```

In [ ]:
# After starting the Docker container, test with the same request:
response = requests.post(
    "http://localhost:5001/invocations",
    json=payload,
)

print(f"Status: {response.status_code}")
print(f"Forecast: {response.json()}")

---
**That's it!** We've gone from experiment tracking to a containerized model serving forecasts via REST API.